`open3d_env`

In [2]:
import numpy as np 
from scipy.spatial import KDTree
import trimesh
import glob
import os
from sklearn.decomposition import PCA
import tqdm
import pickle
import pandas as pd

import sys
sys.path.append('/home/pelissier/These-ATER/Papier_international3/Dataset')  # Adjust the path based on the relative location
from utils import *
sys.path.append('/home/pelissier/These-ATER/Papier_international3/Code')  # Adjust the path based on the relative location
from utils_comparaison import *

In [3]:
dir_mesh_aligned = "/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned"

## ACP

In [4]:
def run_acp(mesh, aff= False):
    "Mesh est un Trimesh"
    # Extract vertices
    vertices = np.array(mesh.vertices)
    center = np.mean(vertices, axis=0)

    # Compute the covariance matrix
    cov_matrix = np.cov(vertices.T)

    # Perform eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Sort eigenvectors by eigenvalues (descending order)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Principal axes are the eigenvectors
    principal_axes = eigenvectors

    # Print results
    if aff:
        print("Eigenvalues:", eigenvalues)
        print("Principal Axes (Eigenvectors):\n", principal_axes)

        # Visualize the principal axes (optional)
        # Each eigenvector is scaled by its eigenvalue for visualization
        for i, axis in enumerate(principal_axes.T):
            print(f"Principal Axis {i + 1}: {axis}")
        
    return eigenvalues, center, eigenvectors

def angle_between_vectors(u, v):
    # Calcul du produit scalaire
    dot_product = np.dot(u, v)
    # Calcul des normes
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    # Calcul de l'angle en radians
    cos_theta = dot_product / (norm_u * norm_v)
    # Clamp pour éviter les erreurs dues aux approximations numériques
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    angle_rad = np.arccos(cos_theta)
    # Conversion en degrés
    angle_deg = np.degrees(angle_rad)
    return angle_deg

def align_principal_axes(mesh1_axes, mesh2_axes, mesh2):
    """
    Align the first principal axes of two meshes while keeping the third axes collinear.
    
    Parameters:
        mesh1_axes (np.ndarray): Principal axes of the first mesh (3x3 matrix).
        mesh2_axes (np.ndarray): Principal axes of the second mesh (3x3 matrix).
        vertices2 (np.ndarray): Vertices of the second mesh (Nx3 matrix).
        
    Returns:
        np.ndarray: Transformed vertices of the second mesh.
    """
    # Extract third principal axes
    v3_1 = mesh1_axes[:, 2]
    v3_2 = mesh2_axes[:, 2]

    # Ensure the third axes are aligned
    # angle = abs(angle_between_vectors(v3_1, v3_2))
    # if ((angle > 30) and ((angle < 150) or (angle > 210))) :
    #     raise ValueError("Third principal axes are not collinear. Pre-align them before using this function.")

    # Project the first axis of mesh1 onto the plane orthogonal to v3_1
    v1_1 = mesh1_axes[:, 0]
    v1_1_proj = v1_1 - np.dot(v1_1, v3_1) * v3_1
    v1_1_proj /= np.linalg.norm(v1_1_proj)

    # Get the first axis of mesh2
    v1_2 = mesh2_axes[:, 0]

    # Compute the rotation angle to align v1_2 with v1_1_proj in the plane orthogonal to v3_1
    cos_theta = np.dot(v1_2, v1_1_proj)
    sin_theta = np.linalg.norm(np.cross(v1_2, v1_1_proj))

    # Compute the rotation matrix around v3_1
    v3_cross = np.array([
        [0, -v3_1[2], v3_1[1]],
        [v3_1[2], 0, -v3_1[0]],
        [-v3_1[1], v3_1[0], 0]
    ])
    rotation_matrix = (
        np.eye(3)
        + sin_theta * v3_cross
        + (1 - cos_theta) * np.dot(v3_cross, v3_cross)
    )

    # Apply the rotation to the vertices of the second mesh
    transformed_vertices2 = np.dot(mesh2.vertices, rotation_matrix.T)
    
    # Create a new mesh with rotated vertices
    aligned_mesh2 = trimesh.Trimesh(vertices=transformed_vertices2, faces=mesh2.faces)

    return aligned_mesh2, rotation_matrix

In [5]:
import numpy as np

def signed_angle_between_vectors(u, v, normal=None):
    """
    Compute the signed angle between two vectors u and v.
    
    Parameters:
        u (array-like): First vector.
        v (array-like): Second vector.
        normal (array-like, optional): Normal vector to determine the sign (used for 3D).
        
    Returns:
        angle_deg (float): Signed angle in degrees from u to v.
    """
    # Calculate the dot product and norms
    dot_product = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    
    # Compute the cosine of the angle
    cos_theta = dot_product / (norm_u * norm_v)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)  # Clamp for numerical stability
    
    # Calculate the angle in radians
    angle_rad = np.arccos(cos_theta)
    
    # Determine the sign of the angle
    if len(u) == 3 and len(v) == 3:  # 3D case
        if normal is None:
            raise ValueError("A normal vector is required for the 3D case to determine the sign.")
        cross_product = np.cross(u, v)
        sign = np.sign(np.dot(cross_product, normal))
    elif len(u) == 2 and len(v) == 2:  # 2D case
        sign = np.sign(u[0]*v[1] - u[1]*v[0])  # Determinant for 2D
    else:
        raise ValueError("Vectors must be 2D or 3D.")
    
    # Apply the sign to the angle
    signed_angle_rad = sign * angle_rad
    
    # Convert to degrees
    signed_angle_deg = np.degrees(signed_angle_rad)
    
    return signed_angle_deg


In [6]:
paths_mesh_iso = files = glob.glob(os.path.join("/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_remeshing_iso/airplane/", "**", "*.obj"), recursive=True); print(len(paths_mesh_iso))

726


In [7]:
## Load the meshes
# Mesh source : par défaut le premier de la liste
path_mesh_source = paths_mesh_iso[0]; print(path_mesh_source)
mesh_source = trimesh.load(path_mesh_source, process=False)
# ACP mesh source
_, _, eigenvectors_source = run_acp(mesh_source, aff=False)
# Save mesh source
directory, name = os.path.split(path_mesh_source)
categorie, type = get_info_path(path_mesh_source)
mesh_source.export(os.path.join(dir_mesh_aligned, categorie, type, name.replace(".obj", "_aligned.obj")))
metadata = {'transformations0': np.eye(4)}            
with open(os.path.join(dir_mesh_aligned, categorie, type, name.replace(".obj", "_aligned.pkl")), "wb") as f: pickle.dump(metadata, f)    
with open("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/Dataset-aligned/"+categorie+"/"+categorie+"_source_mesh.txt", "w") as file: file.write(path_mesh_source)        

# Les autres meshes à traiter 
paths_mesh_to_rotate = paths_mesh_iso[1:]

pbl = []
for path_mesh_to_rotate in tqdm.tqdm(paths_mesh_to_rotate[:0]):
    #try : 
    if True:
        #path_mesh_to_rotate = "/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_remeshing_iso/airplane/train/airplane_0020_SMPLER_centered_scaled_remeshing_iso_iter7.obj"
        #print(path_mesh_to_rotate)
        # Load the meshes
        mesh2_to_rotate = trimesh.load(path_mesh_to_rotate, process=False)
        num = os.path.basename(path_mesh_to_rotate).split("_")[1]
        directory, name = os.path.split(path_mesh_to_rotate)
        categorie, type = get_info_path(path_mesh_to_rotate)
        # ACP mesh_to_rotate
        _, _, eigenvectors_to_rotate = run_acp(mesh2_to_rotate, aff=False)
        
        # Align mesh2 to mesh1
        # ### option 1 : acp
        # aligned_mesh2, matrix = align_principal_axes(eigenvectors_source, eigenvectors_to_rotate, mesh2_to_rotate)
        # aligned_mesh2.export(os.path.join(dir_mesh_aligned, categorie, type, name.replace(".obj", "_aligned.obj")))
        
        ## option 2 : rotation
        matrix = create_rotation_matrix('Z', np.round(signed_angle_between_vectors(eigenvectors_source[:,0], eigenvectors_to_rotate[:,0], normal = np.array([0, 0, 1]))))
        mesh2_to_rotate.apply_transform(matrix)
        mesh2_to_rotate.export(os.path.join(dir_mesh_aligned, categorie, type, name.replace(".obj", "_aligned.obj")))

        # Sauvegarde transformatison
        metadata = {'transformations0': matrix}            
        with open(os.path.join(dir_mesh_aligned, categorie, type, name.replace(".obj", "_aligned.pkl")), "wb") as f: pickle.dump(metadata, f)
            
    # except Exception as e:
    #     pbl.append(path_mesh_to_rotate)
    #     directory, name = os.path.split(path_mesh_to_rotate)
    #     categorie, type = get_info_path(path_mesh_to_rotate)
    #     mesh2_to_rotate.export(os.path.join(dir_mesh_aligned, categorie, type, name.replace(".obj", "_aligned-PBL.obj")))
    #     metadata = {'transformations0': None}            
    #     with open(path_mesh_to_rotate.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_aligned-PBL.pkl'), "wb") as f: pickle.dump(metadata, f)
    

print(len(pbl), pbl)     

/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_remeshing_iso/airplane/test/airplane_0637_SMPLER_centered_scaled_remeshing_iso_iter5.obj


0it [00:00, ?it/s]

0 []


### Debug

In [8]:
# Load the meshes
path_mesh_source = path_mesh_source
path_mesh_target = [path for path in paths_mesh_to_rotate if "0020" in path][0]; print(path_mesh_target)

/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_remeshing_iso/airplane/train/airplane_0020_SMPLER_centered_scaled_remeshing_iso_iter7.obj


In [9]:
# Load the mesh
mesh_target = trimesh.load(path_mesh_target, process=False)
mesh_source = trimesh.load(path_mesh_source, process=False)
mesh_source.apply_translation(np.array([1,1,1]))

# ACP 
eigenvalues_source, center_source, eigenvectors_source = run_acp(mesh_source, aff=False)
eigenvalues_target, center_target, eigenvectors_target = run_acp(mesh_target, aff=False)


# Create a Trimesh scene
scene = trimesh.Scene()
scene.add_geometry(mesh_source)
scene.add_geometry(mesh_target)

# Scale the axes for visualization
axis_length = np.max(eigenvalues_source) * 10  # Scale factor
scaled_axes_source = eigenvectors_source.T * axis_length

# Add principal axes to the scene
colors = [[255, 0, 0, 255], [0, 255, 0, 255], [0, 0, 255, 255]]  # RGBA for X, Y, Z
for i, axis in enumerate(scaled_axes_source):
    start_point = center_source
    end_point = center_source + axis
    line = trimesh.load_path(np.array([start_point, end_point]))
    line.colors = np.array([colors[i]])  # Assign a single color per line
    scene.add_geometry(line)

# Scale the axes for visualization
axis_length = np.max(eigenvalues_target) * 10  # Scale factor
scaled_axes_target= eigenvectors_target.T * axis_length

# Add principal axes to the scene
colors = [[255, 0, 255, 255], [0, 255, 255, 255], [255, 255, 0, 255]]  # RGBA for X, Y, Z
for i, axis in enumerate(scaled_axes_target):
    start_point = center_target
    end_point = center_target + axis
    line = trimesh.load_path(np.array([start_point, end_point]))
    line.colors = np.array([colors[i]])  # Assign a single color per line
    scene.add_geometry(line)
    
    
## test alignement
mesh_target_align = mesh_target.copy()
angle_vect1 = signed_angle_between_vectors(eigenvectors_source[:,0], eigenvectors_target[:,0], normal = np.array([0, 0, 1]))
matrix = create_rotation_matrix('Z', angle_vect1)
mesh_target_align.apply_transform(matrix)
mesh_target_align.apply_translation(np.array([-2,2,2]))
scene.add_geometry(mesh_target_align)

# Show the scene
scene.show()

In [10]:
import trimesh

# Charger les maillages
mesh1 = mesh_source
mesh2 = mesh_target_align

# Vérifiez si les maillages sont des volumes
print(f"Mesh1 est un volume : {mesh1.is_volume}")
print(f"Mesh2 est un volume : {mesh2.is_volume}")

# Réparer les maillages s'ils ne sont pas des volumes
if not mesh1.is_volume:
    mesh1 = mesh1.fill_holes()
    print("Mesh1 a été réparé.")

if not mesh2.is_volume:
    mesh2 = mesh2.fill_holes()
    print("Mesh2 a été réparé.")

# Re-vérifiez après la réparation
print(f"Mesh1 est maintenant un volume : {mesh1.is_volume}")
print(f"Mesh2 est maintenant un volume : {mesh2.is_volume}")

# Calculer l'intersection si les maillages sont maintenant étanches
if mesh1.is_volume and mesh2.is_volume:
    intersection = trimesh.boolean.intersection([mesh1, mesh2], engine='scad')  # Utilisez 'scad' si 'blender' pose problème
    if intersection.is_empty:
        print("Les maillages n'ont pas d'intersection.")
    else:
        print(f"Volume de l'intersection : {intersection.volume} unités cubiques.")
else:
    print("Impossible de calculer l'intersection car les maillages ne sont pas des volumes fermés.")


Mesh1 est un volume : False
Mesh2 est un volume : False


ModuleNotFoundError: No module named 'networkx'

In [ ]:
np.round(signed_angle_between_vectors(eigenvectors_source[:,2], eigenvectors_target[:,2], normal = np.array([0, 0, 1])))

## Orientation ok


In [ ]:
# Matrice de rotation de 180° autour de l'axe Z
rotation_matrix = trimesh.transformations.rotation_matrix(
    angle=np.pi,  # 180° en radians
    direction=[0, 0, 1],  # Axe Z
    point=[0, 0, 0]  # Centre de rotation (origine)
)
print(rotation_matrix.shape)

In [ ]:
mesh_to_rotate = read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/airplane_meshes_not_aligned.txt'); print(len(mesh_to_rotate))
mesh_pbl = read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/airplane_meshes_PBL.txt'); print(len(mesh_pbl))
all_mesh_aligned = read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/airplane_meshes_aligned.txt'); print(len(all_mesh_aligned))

# list_180 = ['0010','0014','0018','0020','0030','0038','0040','0044','0065', '0089', '0099', '0080', '0039', '0019', '0076']
# list_90 = ['0052', '0057', '0067']
# list_moins90 = ['0073', '0056', '0001']

for mesh_path in tqdm.tqdm(all_mesh_aligned[:0]):
    #try:
    if True:
        mesh = trimesh.load(mesh_path.replace('Dataset-aligned', dir_mesh_aligned))
        num = os.path.basename(mesh_path).split("_")[1]; passe = False
        matrix_to_rotate = create_rotation_matrix('Z', 180)
        if mesh_path in mesh_to_rotate: continue
        #     if num in list_180:
        #         matrix_to_rotate = create_rotation_matrix('Z', 180)
        #     elif num in list_90:
        #         matrix_to_rotate = create_rotation_matrix('Z', -90)
        #     elif num in list_moins90:
        #         matrix_to_rotate = create_rotation_matrix('Z', 90)
        #     else : 
        #         passe = True
            
            #if not(passe):
                # Appliquer la rotation
                # mesh.apply_transform(matrix_to_rotate)
                # mesh.export(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.obj'))
                # pkl_mesh = read_pkl(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '.pkl'))
                # pkl_mesh['transformations1'] = matrix_to_rotate     
                #with open(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.pkl'), "wb") as f: pickle.dump(pkl_mesh, f)
                
            
        elif mesh_path in mesh_pbl: continue
            #mesh.export(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_PBL.obj'))
        
        ## le mesh est déjà bien aligné
        else : #continue
            #mesh.export(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.obj'))
            pkl_mesh = read_pkl(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '.pkl'))
            pkl_mesh['transformations1'] = np.eye(4)        
            #with open(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok.pkl'), "wb") as f: pickle.dump(pkl_mesh, f)
            

### Cas particulier

In [ ]:
df_angle_pbl = pd.read_csv('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/rotation_airplane_PBL.csv')
paths_pbl = df_angle_pbl['path'].values; print(len(paths_pbl))
angles = df_angle_pbl['angle'].values; print(len(angles))
df_angle_pbl

In [24]:
# for i in tqdm(tqdm(range(len(paths_pbl)*0))):
#     mesh_path = glob.glob("Dataset-aligned/airplane/*/"+paths_pbl[i], recursive=True)[0]
#     mesh_pbl = trimesh.load(mesh_path.replace('Dataset-aligned', dir_mesh_aligned))
#     matrix_to_rotate_pbl = create_rotation_matrix('Z', angles[i])
#     mesh_pbl.apply_transform(matrix_to_rotate_pbl)
#     mesh_pbl.export(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok-pbl.obj'))
#     pkl_mesh = read_pkl(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '.pkl'))
#     pkl_mesh['transformations1'] = matrix_to_rotate_pbl     
#     with open(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok-pbl.pkl'), "wb") as f: pickle.dump(pkl_mesh, f)

In [29]:
# mesh_path_todo = "Dataset-aligned/airplane/train/airplane_0370_SMPLER_centered_scaled_remeshing_iso_iter5_aligned.obj"
# mesh_todo = trimesh.load(mesh_path_todo.replace('Dataset-aligned', dir_mesh_aligned))
# matrix_todo = create_rotation_matrix('Z', -150)
# mesh_todo.apply_transform(matrix_todo)
# # obj
# mesh_todo.export(mesh_path_todo.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok2.obj'))
# #pkl
# pkl_mesh_todo = read_pkl(mesh_path_todo.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '.pkl'))
# pkl_mesh_todo['transformations1'] = matrix_todo     
# with open(mesh_path_todo.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_ok2.pkl'), "wb") as f: pickle.dump(pkl_mesh_todo, f)

> > > > Normalement, à ce stage tous les meshs sont bien alignés entre eux !!

## Alignement avec US

Car : ModelNet --> Rx -90° --> Ry 180° --> US 

Cup : ModelNet --> Rx -90° --> Ry 180° --> US 

Airplaine : ModelNet --> Rx -90° --> Ry 90° --> US 

In [13]:
mesh_to_rotate = read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/airplane_meshes_aligned_ok_tous.txt'); print(len(mesh_to_rotate))
mesh_pbl = []# read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/Alignement/paths/car_meshes_PBL.txt'); print(len(mesh_pbl))


for mesh_path in tqdm.tqdm(mesh_to_rotate[:0]):
    #print(mesh_path)
    #try:
    if True:
        mesh = trimesh.load(mesh_path.replace('Dataset-aligned', dir_mesh_aligned))
        if mesh_path in mesh_to_rotate:
            matrix1 = create_rotation_matrix('X', -90)
            matrix2 = create_rotation_matrix('Y', 90)
            mesh.apply_transform(matrix1)
            mesh.apply_transform(matrix2)
            # Sauvegarder le mesh transformé
            #mesh.export(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_US.obj'))
            # Sauvegarde transformatison
            mesh_pkl = read_pkl(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '.pkl'))
            mesh_pkl['transformations2'] = matrix1; mesh_pkl['transformations3'] = matrix2 
            #with open(mesh_path.replace('Dataset-aligned', dir_mesh_aligned).replace('.obj', '_US.pkl'), "wb") as f: pickle.dump(mesh_pkl, f)
            #continue


726


100%|██████████| 726/726 [01:14<00:00,  9.77it/s]
